In [4]:
import pandas as pd
import numpy as np
import re
import string
import nltk
nltk.download('punkt')
# !pip install sklearn
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
# ML Libraries
from sklearn.metrics import accuracy_score, classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
# Global Parameters
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\c2099176\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
# Loading the dataset
def load_dataset(type_):
    with open(f"../data/{type_}_text.txt", "r", encoding="utf-8") as fp:
        x = fp.readlines()
    with open(f"../data/{type_}_labels.txt", "r", encoding="utf-8") as fp:
        y = [int(i) for i in fp.readlines()]
    # Making the data dictionary         
    data = {'text': x, 'label': y}
    # Converting the dictionary in to pandas data frame     
    df = pd.DataFrame(data)
    
    return df

In [6]:
# Pre-processing the tweets
def preprocess_tweets(tweet):
    # Converting to lowercase    
    tweet.lower()
    # Remove urls
    tweet = re.sub(r"http\S+|www\S+|https\S+", '', tweet, flags=re.MULTILINE)
    # Remove user @ references and '#' from tweet
    tweet = re.sub(r'\@\w+|\#','', tweet)
    # Remove punctuations
    tweet = tweet.translate(str.maketrans('', '', string.punctuation))
    # Remove stopwords
    tweet_tokens = word_tokenize(tweet)
    filtered_words = [w for w in tweet_tokens if not w in stop_words]
    
    # Stemming and Lemmatizing both are normalization techniques.
    # We will use either of these not both.
    # Stemming is faster than lemmatization    
    
    ps = PorterStemmer()
    stemmed_words = [ps.stem(w) for w in filtered_words]
    #lemmatizer = WordNetLemmatizer()
    #lemma_words = [lemmatizer.lemmatize(w, pos='a') for w in stemmed_words]
    
    return " ".join(stemmed_words)
#     return " ".join(filtered_words)

In [7]:
# Convert Value to String for output
def value_to_string(sentiment):
    if sentiment == 0:
        return "Negative"
    elif sentiment == 1:
        return "Neutral"
    else:
        return "Positive"

In [8]:
# Naive Bayes model definition
# accepts x,y train, val and test vectors
# prints the classification report
# and returns the model object
def naive_bayes_model(X_train, y_train, X_any, y_any):
    # Training Naive Bayes model
    NB_model = MultinomialNB()
    NB_model.fit(X_train, y_train)
    y_predict_nb = NB_model.predict(X_any)
    print(classification_report(y_any, y_predict_nb))
#     print(accuracy_score(y_any, y_predict_nb))
    return NB_model

In [10]:
# Logistic Regression model definition
# accepts x,y train, val and test vectors
# prints the classification report
# and returns the model object
def logistic_reg_model(X_train, y_train, X_any, y_any):
    # Training Logistics Regression model with netown slover and 10000 iterations
    LR_model = LogisticRegression(solver='newton-cg', max_iter=10000, multi_class="multinomial")
    LR_model.fit(X_train, y_train)
    y_predict_lr = LR_model.predict(X_any)
    print(classification_report(y_any, y_predict_lr))
#     print(accuracy_score(y_any, y_predict_lr))
    return LR_model

In [11]:
# Load train dataset
train_dataset = load_dataset("train")
# old_dataset = train_dataset
# Preprocess train data
train_dataset.text = train_dataset['text'].apply(preprocess_tweets)

# Load val dataset
val_dataset = load_dataset("val")
# Preprocess val data
val_dataset.text = val_dataset['text'].apply(preprocess_tweets)

# Load test dataset
test_dataset = load_dataset("test")
# Preprocess data
test_dataset.text = test_dataset['text'].apply(preprocess_tweets)

In [12]:
vector = TfidfVectorizer(sublinear_tf=True)
tf_vector = vector.fit(np.array(train_dataset.iloc[:, 0]).ravel())

In [13]:
# train
X_train = tf_vector.transform(np.array(train_dataset.iloc[:, 0]).ravel())
y_train = np.array(train_dataset.iloc[:, 1]).ravel()

In [15]:
# validate
X_val = tf_vector.transform(np.array(val_dataset.iloc[:, 0]).ravel())
y_val = np.array(val_dataset.iloc[:, 1]).ravel()

In [16]:
# test
X_test = tf_vector.transform(np.array(test_dataset.iloc[:, 0]).ravel())
y_test = np.array(test_dataset.iloc[:, 1]).ravel()
# test_feature = tf_vector.transform(np.array(test_dataset.iloc[:, 0]).ravel())

In [17]:
naive_bayes_model(X_train, y_train, X_val, y_val)

              precision    recall  f1-score   support

           0       0.80      0.01      0.03       312
           1       0.56      0.77      0.65       869
           2       0.67      0.65      0.66       819

    accuracy                           0.60      2000
   macro avg       0.67      0.48      0.44      2000
weighted avg       0.64      0.60      0.55      2000



MultinomialNB()

In [18]:
naive_bayes_model(X_train, y_train, X_test, y_test)

              precision    recall  f1-score   support

           0       0.80      0.05      0.10      3972
           1       0.53      0.84      0.65      5937
           2       0.50      0.52      0.51      2375

    accuracy                           0.53     12284
   macro avg       0.61      0.47      0.42     12284
weighted avg       0.61      0.53      0.44     12284



MultinomialNB()

In [19]:
logistic_reg_model(X_train, y_train, X_val, y_val)

              precision    recall  f1-score   support

           0       0.63      0.38      0.48       312
           1       0.63      0.74      0.68       869
           2       0.73      0.70      0.71       819

    accuracy                           0.67      2000
   macro avg       0.66      0.61      0.62      2000
weighted avg       0.67      0.67      0.66      2000



LogisticRegression(max_iter=10000, multi_class='multinomial',
                   solver='newton-cg')

In [20]:
logistic_reg_model(X_train, y_train, X_test, y_test)

              precision    recall  f1-score   support

           0       0.69      0.38      0.49      3972
           1       0.59      0.75      0.66      5937
           2       0.53      0.56      0.54      2375

    accuracy                           0.59     12284
   macro avg       0.60      0.56      0.57     12284
weighted avg       0.61      0.59      0.58     12284



LogisticRegression(max_iter=10000, multi_class='multinomial',
                   solver='newton-cg')

In [21]:
# RNN Text Classification
# !pip install tensorflow
# !pip install keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical

Using TensorFlow backend.


In [24]:
max_fatures = 2000
max_len = 200

tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(train_dataset['text'].values)
# ////////////////////////
X_train = tokenizer.texts_to_sequences(train_dataset['text'].values)
X_train = pad_sequences(X_train, maxlen=max_len)
# ////////////////////////
X_val = tokenizer.texts_to_sequences(val_dataset['text'].values)
X_val = pad_sequences(X_val, maxlen=max_len)
# ////////////////////////
X_test = tokenizer.texts_to_sequences(test_dataset['text'].values)
X_test = pad_sequences(X_test, maxlen=max_len)

In [25]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim, input_length = X_train.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 200, 128)          256000    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 200, 128)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 591       
Total params: 511,391
Trainable params: 511,391
Non-trainable params: 0
_________________________________________________________________
None


In [26]:
Y_train = pd.get_dummies(train_dataset['label']).values
Y_val = pd.get_dummies(val_dataset['label']).values
Y_test = pd.get_dummies(test_dataset['label']).values
# X_train, X_test, Y_train, Y_test = train_test_split(X_train, Y_train, test_size = 0.33, random_state = 42)
# print(X_train.shape,Y_train.shape)
# print(X_val.shape,Y_val.shape)
# print(X_test.shape,Y_test.shape)

In [28]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 7, validation_data=(X_val, Y_val), batch_size=batch_size, verbose = 2)

Train on 45615 samples, validate on 2000 samples
Epoch 1/7
 - 341s - loss: 0.7470 - accuracy: 0.6653 - val_loss: 0.7459 - val_accuracy: 0.6750
Epoch 2/7
 - 362s - loss: 0.7232 - accuracy: 0.6745 - val_loss: 0.7470 - val_accuracy: 0.6640
Epoch 3/7
 - 333s - loss: 0.7068 - accuracy: 0.6842 - val_loss: 0.7359 - val_accuracy: 0.6775
Epoch 4/7
 - 332s - loss: 0.6947 - accuracy: 0.6908 - val_loss: 0.7468 - val_accuracy: 0.6700
Epoch 5/7
 - 333s - loss: 0.6709 - accuracy: 0.7004 - val_loss: 0.7415 - val_accuracy: 0.6795
Epoch 6/7
 - 334s - loss: 0.6554 - accuracy: 0.7084 - val_loss: 0.7650 - val_accuracy: 0.6710
Epoch 7/7
 - 335s - loss: 0.6393 - accuracy: 0.7146 - val_loss: 0.7718 - val_accuracy: 0.6670


In [29]:
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

score: 0.99
acc: 0.57


In [30]:
tweet = ['i am glad the weather is sunny!']
# vectorizing the tweet by the pre-fitted tokenizer instance
tweet = tokenizer.texts_to_sequences(tweet)
# padding the tweet to have exactly the same shape as `embedding` input
tweet = pad_sequences(tweet, maxlen=max_len, dtype='int32', value=0)
# print(tweet)
sentiment = model.predict(tweet, batch_size=1, verbose = 2)[0]
output = value_to_string(np.argmax(sentiment))
print(output)

Positive
